# RA2CE feature: Losses analysis

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import rasterio

from ra2ce.ra2ce_handler import Ra2ceHandler #import the ra2cehandler to run ra2ce analyses

## Single link losses Analysis

In [ ]:
root_dir = Path(".\\data\\single_link_losses")

network_ini = root_dir / "network.ini"
analysis_ini = root_dir / "analysis.ini"

handler = Ra2ceHandler(network=network_ini, analysis=analysis_ini)
criticality_analysis_weight = handler.input_config.analysis_config.config_data.indirect[0].weighing.config_value

handler.configure()
handler.run_analysis()